In [1]:
import graphAlgorithms as ga

/usr/lib/python3.6/importlib/_bootstrap.py:219: UserWarning: To obtain optimal results install the Cython 'munkres' module at  https://github.com/jfrelinger/cython-munkres-wrapper
  return f(*args, **kwds)


Examples on how to use different community detection algorithms 

# Load Network

In [2]:
#location where the raw data files are stored
# TODO: change to package location 
graph_location = "/mnt/data/apavel/graphalgo/testset/"

In [3]:
#location where output should be saved
location = "/mnt/data/apavel/graphalgo/testoutput/"

In [4]:
import glob
import pandas as pd
import networkx as nx
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

In [5]:

labels = []
networks_graphs = []
cnt = 0
print("load networks")
#gets all files located in the specified folder that end on .edgelist
for path in glob.glob(graph_location +"*.edgelist"):
    if cnt < 1:
        #you can specify that only part of the file name should be used as network name for later identification
        name =  path.split("/")[-1].replace(".rds.edgelist", "")


        #read the edgelist file as a dataframe
        fh = pd.read_csv(path, sep="\t")
        #convert it into a NetworkX graph G and specify the column names of the node pairs
        G=nx.from_pandas_edgelist(fh, "V1", "V2")

        #if you have an unweighted network assign all edges the same edge weight - here a value of 1 is assigned
        for u, v, d in G.edges(data=True):
            d['weight'] = 1


        #save the graph objects to a list (only suitable if small networks are processed)
        #this is the main objects used for the examples below, which contains all networks
        networks_graphs.append(G)
        labels.append(name)




        print("loaded", name)
    cnt = cnt + 1

load networks
loaded lestaurtinib


In [6]:
nodes = []
for net in networks_graphs:
    for node in net.nodes():
        if node not in nodes:
            nodes.append(node)

## Some unweighted community detection algorithms

All communiy detection algorithms return a dict, where keys are node IDs and values are list of communities that node belongs to. In case of overlapping communities this may be more than one community.

In [7]:
asy = ga.communities.async_fluid(networks_graphs[0], k=10, return_object=False)

In [8]:
label = ga.communities.label_propagation(networks_graphs[0], return_object=False)

In [9]:
info = ga.communities.infomap(networks_graphs[0], return_object=False)

In [10]:
walktrap = ga.communities.walktrap(networks_graphs[0], return_object=False)

## Some weighted community detection algorithms

In [12]:
mod =  ga.communities.greedy_modularity(networks_graphs[0], weights="weight", return_object=False)

In [13]:
leiden = ga.communities.leiden(networks_graphs[0], weights="weight", return_object=False)

In [19]:
louvain = ga.communities.louvain(networks_graphs[0], weights="weight", return_object=False)

## Ensembl community detection

The ensembl algorithm is based on an adapted implementation of: https://arxiv.org/pdf/1902.04014.pdf
Here we want to create an ensembl out of labelpropagation, infomap and the louvain algorithm.

TODO: put proper citation
TODO: maybe for case study take x best performing algorithms based on the below metrics (or others)

In [15]:
community_list=[label, info, louvain]

Only algorithms that are not requireing connected graphs can be provided as selected algorithms

In [16]:
algorithms = [ga.communities.label_propagation,ga.communities.infomap, ga.communities.louvain ]
parameters = [{"return_object": False}, {"return_object": False},{"weights":"weight","return_object": False}]

In [18]:
consensuscommunities, cons1, consgraph  = ga.communities.fast_consensus(networks_graphs[0], communities = community_list, algorithms=algorithms, parameters=parameters, thresh=0.5, max_iter=10)

start fast consensus
no initial graph provided create from communities
create consensus graph object
transforming initial consensus graph in its communities based on selected treshold
initial removed edges 445828
giving initial graph free
start convergence on  reduced graph
estimating communities  label_propagation
estimating communities  infomap
estimating communities  louvain
create consensus graph object
estimating communities  label_propagation
estimating communities  infomap
estimating communities  louvain
create consensus graph object


## Overlapping Communities

In [20]:
angel = ga.communities.angel(networks_graphs[0], return_object=False)

## Some community evaluation metrics

In [31]:
community_list=[asy, label, info, walktrap,  mod, leiden, louvain, consensuscommunities, angel]
algo_labels=["async_fluids","labelpropagation", "infomap", "walktrap",  "modularity", "leiden", "louvain", "consensuscommunities", "angel"]

How many communities are detected by the algorithms?

In [32]:
for i in range(len(community_list)):
    print("algo ", algo_labels[i], "detects ", ga.communities.get_number_of_communities(community_list[i]), "communities")

algo  async_fluids detects  10 communities
algo  labelpropagation detects  32 communities
algo  infomap detects  41 communities
algo  walktrap detects  24 communities
algo  modularity detects  9 communities
algo  leiden detects  12 communities
algo  louvain detects  12 communities
algo  consensuscommunities detects  28 communities
algo  angel detects  14 communities


What is the community size distribution?

In [33]:
for i in range(len(community_list)):
    print("algorithm ", algo_labels[i], "has a community size distribution of:")
    print(ga.communities.get_number_of_nodes_community(community_list[i], in_detail=False))

algorithm  async_fluids has a community size distribution of:
{'mean': 97.7, 'median': 99.5, 'std': 44.828190535272185, 'kurt': -0.8650795697808324, 'skewness': -0.18523195442603282}
algorithm  labelpropagation has a community size distribution of:
{'mean': 30.53125, 'median': 13.5, 'std': 33.69118232014175, 'kurt': -0.04357752886502286, 'skewness': 1.1521664522131856}
algorithm  infomap has a community size distribution of:
{'mean': 23.829268292682926, 'median': 14, 'std': 25.52636131435931, 'kurt': 1.6974896786197577, 'skewness': 1.628958776965327}
algorithm  walktrap has a community size distribution of:
{'mean': 40.708333333333336, 'median': 16.5, 'std': 43.055554776375246, 'kurt': -0.7912855475008698, 'skewness': 0.8215536023375906}
algorithm  modularity has a community size distribution of:
{'mean': 108.55555555555556, 'median': 115, 'std': 68.92951311142258, 'kurt': 0.026649701566436423, 'skewness': 0.5380218321499421}
algorithm  leiden has a community size distribution of:
{'me

What is the average internal degree?  The scores for the individual communities are returned in the sc object

In [34]:
for i in range(len(community_list)):
    print("algo ", algo_labels[i], "has a community internal degree distribution of:")
    sc, dist = ga.communities.average_internal_degree(community_list[i], networks_graphs[0])
    print(dist)

algo  async_fluids has a community internal degree distribution of:
{'mean': 29.551507857391336, 'median': 23.80774647887324, 'std': 17.498190334556238, 'kurt': 0.306004490490714, 'skewness': 0.9878363790432299}
algo  labelpropagation has a community internal degree distribution of:
{'mean': 15.845698250911369, 'median': 8.915441176470587, 'std': 17.4890472970971, 'kurt': 2.747033880588897, 'skewness': 1.7722517816901706}
algo  infomap has a community internal degree distribution of:
{'mean': 15.20430782181294, 'median': 9.142857142857142, 'std': 17.093006638253772, 'kurt': 4.267686483419309, 'skewness': 2.087815928721599}
algo  walktrap has a community internal degree distribution of:
{'mean': 17.62062384158554, 'median': 12.257918552036198, 'std': 17.108046614487332, 'kurt': 0.8599734411515914, 'skewness': 1.2126369028092066}
algo  modularity has a community internal degree distribution of:
{'mean': 30.822651862390188, 'median': 25.222672064777328, 'std': 16.525690810108216, 'kurt': 

What is the internal edge density?  The scores for the individual communities are returned in the sc object

In [35]:
for i in range(len(community_list)):
    print("algo ", algo_labels[i], "has a community internal edge density distribution of:")
    sc, dist = ga.communities.internal_edge_density(community_list[i], networks_graphs[0])
    print(dist)

algo  async_fluids has a community internal edge density distribution of:
{'mean': 0.08018445814403682, 'median': 0.07679640895224721, 'std': 0.030103565217978155, 'kurt': -0.3835523975230428, 'skewness': 0.47553611834063375}
algo  labelpropagation has a community internal edge density distribution of:
{'mean': 0.17073892659168374, 'median': 0.17435576655760143, 'std': 0.05689929702520839, 'kurt': -1.175828866104495, 'skewness': -0.024957808096507245}
algo  infomap has a community internal edge density distribution of:
{'mean': 0.1807036190368201, 'median': 0.17582417582417584, 'std': 0.04254862906856415, 'kurt': -0.685265433337134, 'skewness': 0.12949839982075184}
algo  walktrap has a community internal edge density distribution of:
{'mean': 0.16503485811384136, 'median': 0.159375, 'std': 0.0691588372873254, 'kurt': -1.473767664895824, 'skewness': -0.02456888066235225}
algo  modularity has a community internal edge density distribution of:
{'mean': 0.10082204574967965, 'median': 0.094

What is the partitions modularity?  The scores for the individual communities are returned in the sc object

In [36]:
for i in range(len(community_list)):
    print("algorithm ", algo_labels[i], "has a modularity of ", ga.communities.community_modularity(community_list[i], networks_graphs[0]))

algorithm  async_fluids has a modularity of  0.8026040120407566
algorithm  labelpropagation has a modularity of  0.8106671295350322
algorithm  infomap has a modularity of  0.79679892315697
algorithm  walktrap has a modularity of  0.8131041399053339
algorithm  modularity has a modularity of  0.7969897046897019
algorithm  leiden has a modularity of  0.8220075404290097
algorithm  louvain has a modularity of  0.820788370432317
algorithm  consensuscommunities has a modularity of  0.8127982035764162
algorithm  angel has a modularity of  0.5683292114736865


What is the fraction of weak members within a community?  The scores for the individual communities are returned in the sc object

In [37]:
for i in range(len(community_list)):
    print("algo ", algo_labels[i], "has a community fraction of weak members distribution of:")
    sc, dist = ga.communities.fraction_of_weak_members(community_list[i], networks_graphs[0])
    print(dist)

algo  async_fluids has a community fraction of weak members distribution of:
{'mean': 0.03497028600809179, 'median': 0.014084507042253521, 'std': 0.047773429006116405, 'kurt': -0.3361875849740361, 'skewness': 1.0645319349956877}
algo  labelpropagation has a community fraction of weak members distribution of:
{'mean': 0.08542527801274852, 'median': 0.0, 'std': 0.18337730746705894, 'kurt': 5.110104135135716, 'skewness': 2.3739395322246137}
algo  infomap has a community fraction of weak members distribution of:
{'mean': 0.14628442714921544, 'median': 0.0, 'std': 0.25151576268368314, 'kurt': 2.7501898721776934, 'skewness': 1.8851415690242208}
algo  walktrap has a community fraction of weak members distribution of:
{'mean': 0.03709127377333845, 'median': 0.0, 'std': 0.08385645428568365, 'kurt': 6.030799965048608, 'skewness': 2.6807336970331788}
algo  modularity has a community fraction of weak members distribution of:
{'mean': 0.01234805922931076, 'median': 0.007246376811594203, 'std': 0.01

What is the cut ratio?  The scores for the individual communities are returned in the sc object

In [38]:
for i in range(len(community_list)):
    print("algo ", algo_labels[i], "has a community cut ratio distribution distribution of:")
    sc, dist = ga.communities.cut_ratio(community_list[i], networks_graphs[0])
    print(dist)

algo  async_fluids has a community cut ratio distribution distribution of:
{'mean': 0.09186704098575485, 'median': 0.07652027220766985, 'std': 0.0753473476342785, 'kurt': 0.11872445049245917, 'skewness': 1.1221134783452948}
algo  labelpropagation has a community cut ratio distribution distribution of:
{'mean': 0.2502691946252538, 'median': 0.2150960874965954, 'std': 0.1771679949281321, 'kurt': -1.2196384979015908, 'skewness': 0.3349959587386101}
algo  infomap has a community cut ratio distribution distribution of:
{'mean': 0.2948050407647503, 'median': 0.28629534323898065, 'std': 0.20306551585655716, 'kurt': 0.753198620578793, 'skewness': 0.8949435964656937}
algo  walktrap has a community cut ratio distribution distribution of:
{'mean': 0.19270209145093384, 'median': 0.13381889191562857, 'std': 0.15485942735687633, 'kurt': -0.7259038318065767, 'skewness': 0.7907887172193473}
algo  modularity has a community cut ratio distribution distribution of:
{'mean': 0.0637376481683399, 'median': 

What is the community density w.r.t. the whole graph density?  The scores for the individual communities are returned in the sc object

In [39]:
for i in range(len(community_list)):
    print("algo ", algo_labels[i], " has a community density w.r.t. the graph density distribution of:")
    sc, dist = ga.communities.community_density_to_graph(community_list[i], networks_graphs[0])
    print(dist)

algo  async_fluids  has a community density w.r.t. the graph density distribution of:
{'mean': 8.877799759902768, 'median': 8.502684394686007, 'std': 3.332982852915356, 'kurt': -0.3835523975230428, 'skewness': 0.4755361183406333}
algo  labelpropagation  has a community density w.r.t. the graph density distribution of:
{'mean': 18.90373816305988, 'median': 19.304184605228905, 'std': 6.299731608357796, 'kurt': -1.175828866104495, 'skewness': -0.024957808096506037}
algo  infomap  has a community density w.r.t. the graph density distribution of:
{'mean': 20.00700114250193, 'median': 19.466762787285283, 'std': 4.710865642448475, 'kurt': -0.685265433337134, 'skewness': 0.12949839982075229}
algo  walktrap  has a community density w.r.t. the graph density distribution of:
{'mean': 18.27219959642028, 'median': 17.64555587808418, 'std': 7.657073743164437, 'kurt': -1.4737676648958247, 'skewness': -0.024568880662352167}
algo  modularity  has a community density w.r.t. the graph density distributio

What is the hub dominance?  The scores for the individual communities are returned in the sc object

In [40]:
for i in range(len(community_list)):
    print("algo ", algo_labels[i], " has a community hub dominance distribution of:")
    sc, dist = ga.communities.hub_dominace(community_list[i], networks_graphs[0])
    print(dist)

algo  async_fluids  has a community hub dominance distribution of:
{'mean': 0.6532715147776632, 'median': 0.6515151515151515, 'std': 0.15522276906914229, 'kurt': 0.36577659716164757, 'skewness': -0.3970110190906123}
algo  labelpropagation  has a community hub dominance distribution of:
{'mean': 0.9232125220446711, 'median': 0.9676113360323887, 'std': 0.1010448004611136, 'kurt': 2.8209385581724584, 'skewness': -1.5830126491807264}
algo  infomap  has a community hub dominance distribution of:
{'mean': 0.9505117248488609, 'median': 1.0, 'std': 0.06794005137722671, 'kurt': -0.36300198103832493, 'skewness': -1.0132369542704824}
algo  walktrap  has a community hub dominance distribution of:
{'mean': 0.8776202386553007, 'median': 0.90625, 'std': 0.14006267660640123, 'kurt': -0.3384657303462677, 'skewness': -0.8842362533050036}
algo  modularity  has a community hub dominance distribution of:
{'mean': 0.6670466157223834, 'median': 0.7142857142857143, 'std': 0.23588587568795458, 'kurt': -0.42926

There are many more evaluation metrics implemented, which you find more information about in the documentation.

You can for example select the community detection algorithm that performs on average the best on a multitude of selected evaluation metrics or select metrics that are focused on the commmunity types that are needed for your experiments.